# 8. Gyakorlat: Objektum Detekció
## Könyvtárak

In [1]:
import requests
import os
from zipfile import ZipFile

# Function to download a file from a given URL
def download_file(url, dest_folder, file_name):
    if not os.path.exists(dest_folder):  # Create destination directory if it does not exist
        os.makedirs(dest_folder)

    response = requests.get(url, stream=True)
    file_path = os.path.join(dest_folder, file_name)

    with open(file_path, 'wb') as file:  # Open file to write in binary mode
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive chunks
                file.write(chunk)
    return file_path

# Function to extract a zip file into a given directory
def extract_zip(zip_path, extract_to):
    with ZipFile(zip_path, 'r') as zip_ref:  # Open the zip file
        zip_ref.extractall(extract_to)  # Extract all the contents into the directory

# URLs (these are placeholders and need to be replaced with actual URLs)
image_url = 'http://kitti_url_to_download_images.zip'
label_url = 'http://kitti_url_to_download_labels.zip'

# Download and extract images
images_zip_path = download_file(image_url, 'kitti_dataset', 'images.zip')  # Download images zip
extract_zip(images_zip_path, 'kitti_dataset/images')  # Extract images

# Download and extract labels
labels_zip_path = download_file(label_url, 'kitti_dataset', 'labels.zip')  # Download labels zip
extract_zip(labels_zip_path, 'kitti_dataset/labels')  # Extract labels

# Remove the zip files
os.remove(images_zip_path)
os.remove(labels_zip_path)

print("Download and extraction complete.")


ConnectionError: HTTPConnectionPool(host='kitti_url_to_download_images.zip', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x10df0acd0>: Failed to resolve 'kitti_url_to_download_images.zip' ([Errno 8] nodename nor servname provided, or not known)"))